## Co-factor swap

#### Co-factors:

NAD+ / NADH: Nicotinamide adenine dinucleotide (NAD+) is reduced to NADH in the conversion of isoamyl aldehyde to isoamyl alcohol. This is an oxidation-reduction reaction where NAD+ acts as an oxidizing agent.

CoASH / Acetyl-CoA: Coenzyme A (CoASH) is used in the formation of acetyl-CoA, which is a substrate for the reaction that synthesizes isoamyl acetate. Acetyl-CoA is a key molecule in metabolic processes and is formed from acetate and CoASH with the input of energy from ATP.

ATP / AMP + PPi: Adenosine triphosphate (ATP) is converted to adenosine monophosphate (AMP) and inorganic pyrophosphate (PPi) in the reaction that activates acetate to acetyl adenylate as a precursor to acetyl-CoA formation.

In the code the following was done:
- Model yeast8 was loaded
- The objective was set to maximize the production of isoamyl acetate
- The theoretical yield was set to production of isoamyl acetate divided by the uptake of glucose
- The algorithm was run and the result was visualized

In [3]:
pip install cobra

Note: you may need to restart the kernel to use updated packages.


In [5]:
# fix the cameo import errors
from collections.abc import Mapping
import os
import pandas as pd
import matplotlib.pyplot as plt
from cameo.core import manipulation
from cameo import fba, flux_variability_analysis
from cobra.io import read_sbml_model
from cameo import load_model
from matplotlib import pyplot as plt
from cameo.flux_analysis.analysis import flux_variability_analysis as fva
from cameo.util import TimeMachine
from cobra.core.gene import GPR
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
# from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization

In [6]:
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization

In [7]:
model = read_sbml_model("Models/yeast8.xml")

In [8]:
# Choose the reaction that produces isoamyl acetate as the objective (r_1180 transports the product to the extracellular medium).
objective_reaction = model.reactions.r_1180
model.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')

Objective set to maximize r_1180: s_0927 --> s_0928


In [16]:
print(model.reactions.r_0656)

r_0656: s_0803 + s_0927 --> s_0362 + s_0794 + s_0929


Reactions regarding isoamyl acetate:

r_0160(alcohol acetyltransferase (isoamyl alcohol)):  s_0373 (acetyl-CoA) + s_0929 (isoamyl alcohol) --> s_0529 (CoA) + s_0927 (isoamyl acetate)

r_0180 (aldehyde dehydrogenase (isoamyl alcohol, NAD)): s_0236(3-methyilbutanal) + s_0799 (glyoxylate)+ s_1205 (NADH) --> s_0931 (isoamylol) + s_1200 (NAD)

r_0656 (isoamyl acetate-hydrolyzing esterase): eliminates isoamyl acetate

r_1180 --> Transports isoamyl acetate to the outside 


Now use NADP and NADH instead of NAD and NADH. The reaction occurs in the m compartment

MIRAR SI LO DE COMPARTIMENTO N ES VERDAD O MENTIRA

In [17]:
if len(population) >= sample_size:
    sample = numpy.random.choice(population, size=sample_size, replace=False)
else:
    print("Population size must be larger than the sample size when 'replace=False'.")


NameError: name 'population' is not defined

In [81]:
Fluxes=model.optimize()
model.objective = model.reactions.r_1180
py = product_yield(model.reactions.r_1180, model.reactions.r_0180)
swap_optimization = CofactorSwapOptimization(model=model, objective_function=py, plot=False, cofactor_id_swaps=(["s_1200","s_1210"],["s_1205","s_1214"]))


TypeError: object.__init__() takes exactly one argument (the instance to initialize)

In [70]:
#                                                                                                                 NAD (m) NADP(+)(m)  NADH    NADPH(m)                                
swap_optimization = CofactorSwapOptimization(model=model, objective_function=py, plot=False, cofactor_id_swaps=(["s_1200","s_1210"],["s_1205","s_1214"]))

In [85]:
%%time
result = swap_optimization.run(max_size=3.9999999)

Starting optimization at Wed, 15 Nov 2023 16:05:41


HBox()

Finished after 00:00:00
CPU times: total: 15.6 ms
Wall time: 49.1 ms


In [86]:
solution

,index,targets,fitness
